In [19]:
import pandas as pd

In [26]:
data_with_similarity_dir = "./Data/Similarity/"
data_selection_dir = "./Data/Selection/"
import_data_name = "data_explanation_prep_4.pickle"

#data_with_similarity_dir = "./Data/Similarity/sumeczech-1.0-dev_splitted.pickle"

percentage_outlier = 0  # if 0 thresh applied, else remove top outliers
thresh = [0,1.1]  #  [0.49,0.55] [0.46,0.54]
method = "LOF_base"
IsClaimAdded = True


#min_no_sentence_source_text = 5 
#deleted_amount_in_decimal = 0.2 

#### Low LOF -> inlier 
#### High LOF -> outlier
#### Low average cosine distance -> inlier
#### High average cossine -> outlier

In [21]:
data_sim = pd.read_pickle(data_with_similarity_dir + import_data_name) # data_explanation_prep_shortExplanation_prep.pickle"
data_sim['source_text_sentences'] = data_sim['source_text_sentences'].astype(str)
len(data_sim)

465236

In [22]:
# TODO try hybrid model if both outlier and inlier are working perfect are working perfect 

In [23]:
def generate_data_without_outliers(data,outlier_col_name,threshold,saving_path, percentage): # outliers , inliers, hybrid
    
    data_labels_no = len(data.id.unique())
    data_sentences = len(data)
    
    if percentage != 0:
        
        top_highest_inliers = round((1 - percentage) * len(data_sim))
        # select top most inlier sentences 
        selection = data.sort_values([outlier_col_name], ascending=[True]).head(top_highest_inliers)
        # set right order of sentences
        selection = selection.sort_values(['id', 'source_text_sentences_index'], ascending=[True, True])
        # info
        #percentage_outliers = round(((len(selection)/data_sentences)*100),3)
        #print("Deleted {} %  of outlier sentences !".format(percentage_outliers))
        
    else:
   
        selection = data[(data[outlier_col_name] > threshold[0]) & (data[outlier_col_name] < threshold[1])].sort_values(["id", 
                                                                                                                         "source_text_sentences_index"
                                                                                                                        ], ascending = (True, True))
        ouliers = data[(data[outlier_col_name] >= threshold[1])]
        inliers = data[(data[outlier_col_name] <= threshold[0])]

        percentage_outliers = round(((len(ouliers)/data_sentences)*100),3)
        percentage_inliers = round(((len(inliers)/data_sentences)*100),3)

        print("Deleted {} %  of outlier sentences !".format(percentage_outliers))
        print("Deleted {} %  of inlier sentences !".format(percentage_inliers))
    
    selection_labels_no = len(selection.id.unique())
    selection_sentences = len(selection)
    
    if data_labels_no != selection_labels_no:
        print("Some labels are deleted, will not continue, try to increase threshold !")
        return None 
    else:
        difference = round((((data_sentences - selection_sentences)/data_sentences)*100),3)
        print("Deleted {} %  of sentences !".format(difference))
        
        shorter = (
                   selection
                           .groupby(["id"])["source_text_sentences"]
                           .apply(' '.join)
                           .to_frame()
                           .rename(columns={"source_text_sentences": "source_text_shorter"})
                  )
        
        selection = pd.merge(selection[["id","source_text","target_text","statement_prep"]].drop_duplicates(), shorter, on = "id", how = "left") # type
        if IsClaimAdded == True:
            selection['source_text'] = selection['statement_prep'] + selection['source_text']
            selection['source_text_shorter'] = selection['statement_prep'] + selection['source_text_shorter']
            
            if percentage != 0:
                selection.to_pickle(saving_path + "claim_{}_{}_{}".format(outlier_col_name, percentage, import_data_name))
            else:
                selection.to_pickle(saving_path + "claim_{}_{}_{}_{}_{}".format(outlier_col_name,threshold[0],threshold[1],difference,import_data_name))
                
        else:
            
            if percentage != 0:
                selection.to_pickle(saving_path + "{}_{}_{}".format(outlier_col_name, percentage, import_data_name))
            else:
                selection.to_pickle(saving_path + "claim_{}_{}_{}_{}_{}".format(outlier_col_name,threshold[0],threshold[1],difference,import_data_name))
            
        
        print("Data successfully saved!")
        print(len(selection))
        
        #display(selection)

In [24]:
data_sim[["LOF_base"]].describe()

,LOF_base
count,465236.000000
mean,0.529973
std,0.035681
min,0.456932
25%,0.503560
50%,0.519462
75%,0.548292
max,1.000000


In [25]:
#min 475
generate_data_without_outliers(
                                data_sim,
                                method, # None for hybrid 
                                thresh, # list for hybrid
                                data_selection_dir,
                                percentage_outlier
)

Deleted 0.002 %  of outlier sentences !
Deleted 0.0 %  of inlier sentences !
Deleted 0.002 %  of sentences !
Data successfully saved!
12891


In [7]:
# min 46
generate_data_without_outliers(
                                data_sim,
                                method, # None for hybrid 
                                thresh, # list for hybrid
                                data_selection_dir
)

Deleted 16.845 %  of outlier sentences !
Deleted 0.003 %  of inlier sentences !
Deleted 16.848 %  of sentences !
Data successfully transformed!
Data successfully saved!
211863


In [7]:
generate_data_without_outliers(
                                data_sim,
                                method, # None for hybrid 
                                thresh, # list for hybrid
                                data_selection_dir
)

Deleted 16.845 %  of outlier sentences !
Deleted 0.029 %  of inlier sentences !
Deleted 16.874 %  of sentences !
Data successfully transformed!
Data successfully saved!
211863


In [ ]:
data_selection_dir